## 1. Data Loading

We load all preprocessed subreddit submission files (parquet parts) produced from the original `.zst` dumps.  
This step concatenates all parts into a single DataFrame to facilitate uniform cleaning and filtering.


In [1]:
from pathlib import Path
from glob import glob
import pandas as pd

# Resolve project root whether this notebook is opened from /notebooks or repo root
CWD = Path.cwd()
ROOT = CWD if (CWD / "data").exists() else CWD.parent

# Directory where the submission parquet parts were written
SUB_DIR = ROOT / "data" / "processed" / "submissions"

# Collect all parquet parts across included subreddits
paths = sorted(glob(str(SUB_DIR / "*.parquet")))
print(f"Found parquet parts: {len(paths)} files")

# Load in chunks to be memory-friendly
dfs = [pd.read_parquet(p) for p in paths]
df = pd.concat(dfs, ignore_index=True)

# Quick peek at raw columns
df.head(3)

Found parquet parts: 77 files


,id,subreddit,created_utc,title,selftext,score,num_comments,author,url,permalink
0,6qxh3,StockMarket,1215605912,Forex Trading Styles: Fundamental analysis - T...,,1,1,marketstock,http://www.marketstock.net/2008/07/forex-tradi...,/r/StockMarket/comments/6qxh3/forex_trading_st...
1,6qxh8,StockMarket,1215605967,How to invest your first 100$ in the stock market,,3,1,marketstock,http://www.marketstock.net/2008/06/how-to-inve...,/r/StockMarket/comments/6qxh8/how_to_invest_yo...
2,7tpzv,StockMarket,1233351595,"How to create and test a trading strategy, Par...",,1,0,strafefire,http://yousuckattrading.com/2009/01/how-to-cre...,/r/StockMarket/comments/7tpzv/how_to_create_an...


## 2. Timestamp Normalization and Period Restriction (2022–Present)

The `created_utc` field may appear as integers (Unix seconds) or strings.  
We normalize it to timezone-aware UTC timestamps using a two-pass strategy:
1) parse as Unix seconds;
2) for failures, parse generically as datetime.

We then restrict the sample to posts dated from **2022-01-01** to the most recent available date and display the retained date range.

In [2]:
import pandas as pd

def to_utc_ts(series: pd.Series) -> pd.Series:
    """
    Robustly convert 'created_utc' to timezone-aware UTC timestamps.
    Tries epoch seconds first; falls back to generic parsing.
    """
    s1 = pd.to_datetime(series, unit="s", errors="coerce", utc=True)
    s2 = pd.to_datetime(series, errors="coerce", utc=True)
    return s1.fillna(s2)

# Normalize timestamps
df["created_utc"] = to_utc_ts(df["created_utc"])
df = df.dropna(subset=["created_utc"])

# Filter: keep 2022-01-01 and later
DATE_MIN = pd.Timestamp("2022-01-01", tz="UTC")
df = df[df["created_utc"] >= DATE_MIN].copy()

# Show retained date range
print("Retained date range:")
print(df["created_utc"].min(), "→", df["created_utc"].max())

Retained date range:
2022-01-01 00:01:45+00:00 → 2024-12-31 23:58:51+00:00


## 3. Duplicate Removal

We remove duplicate posts to ensure each Reddit submission is represented once.  
Deduplication is performed on the unique submission identifier `id`.  
We report the number of rows removed and the number remaining.

In [3]:
before = len(df)
df = df.drop_duplicates(subset=["id"], keep="last")
after = len(df)

print(f"Duplicates removed: {before - after}")
print(f"Rows remaining:     {after}")

Duplicates removed: 5
Rows remaining:     854855


## 4. Column Selection and Content Cleaning

For downstream analysis we retain only the following variables:

- `subreddit`: the community of origin (categorical context);
- `timecreated`: a *date-only* version of the timestamp (no intra-day time);
- `title`: the textual signal to be used in NLP.

We remove posts whose title is exactly **“[deleted by user]”** (case-insensitive), as these do not contain analyzable content.

In [4]:
# Ensure title is a string and not missing
df["title"] = df["title"].fillna("").astype(str)

# Remove rows where the title is exactly "[deleted by user]" (case-insensitive, trimmed)
mask_deleted_by_user = df["title"].str.strip().str.lower().eq("[deleted by user]")
df = df[~mask_deleted_by_user].copy()

# Create a date-only column (no time-of-day)
df["timecreated"] = df["created_utc"].dt.date

# Keep only requested columns
df_final = df[["subreddit", "timecreated", "title"]].reset_index(drop=True)

# Sanity peek
df_final.head(5)

,subreddit,timecreated,title
0,StockMarket,2022-01-01,Happy 2022 year!
1,StockMarket,2022-01-01,I did the meme! 👁👄👁
2,StockMarket,2022-01-01,Mercedes Benz EQS 2022 ( Electric Car ) Hits t...
3,StockMarket,2022-01-01,WALL STREET
4,StockMarket,2022-01-01,2022 investing tips and goals.


## 6. Summary Checks 

We provide basic descriptive checks to verify the final structure and content distribution.  
These checks are optional but facilitate quick validation of preprocessing outcomes.

In [5]:
print("Unique subreddits:", df_final["subreddit"].nunique())
print(df_final["subreddit"].value_counts().head(10))
print("Date range:", df_final["timecreated"].min(), "→", df_final["timecreated"].max())
df_final.sample(5, random_state=42)

Unique subreddits: 5
subreddit
wallstreetbets    538022
stocks            105956
investing         102895
StockMarket        64248
finance            34469
Name: count, dtype: Int64
Date range: 2022-01-01 → 2024-12-31


,subreddit,timecreated,title
25447,StockMarket,2022-12-31,Cenntro stock ?
403010,wallstreetbets,2022-06-15,Bullish on getting fucked
211869,stocks,2022-03-13,What to do about SEV?
211050,stocks,2022-03-08,Why are CNN Futures so different?
258687,stocks,2023-07-23,Check How Optimized Your Portfolio Is?


In [7]:
# SAVE FINAL CLEAN DATASET (2022 -> most recent)
from pathlib import Path

# create dedicated folder for cleaned datasets
CLEAN_DIR = ROOT / "data" / "processed" / "clean"
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

# final filename (very clear + version stable)
OUT_PATH = CLEAN_DIR / "reddit_posts_minimal_2022plus.parquet"

df.to_parquet(OUT_PATH, index=False)

print("✅ Final cleaned dataset saved here:")
print(str(OUT_PATH))
print("Number of rows:", len(df))

✅ Final cleaned dataset saved here:
/Users/lucasvercauteren/Desktop/passion-project/return-supervised-nlp-finance/data/processed/clean/reddit_posts_minimal_2022plus.parquet
Number of rows: 845590
